In [ ]:
# upload kaggle.json
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle (2).json


In [ ]:
# create kaggle hidden file in root directory and copy kaggle.json there
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# download kaggle dataset
! kaggle datasets download jacobvs/ddos-attack-network-logs

ddos-attack-network-logs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzip kaggle datsaet
! unzip ddos-attack-network-logs.zip

Archive:  ddos-attack-network-logs.zip
replace final-dataset-short.arff? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final-dataset-short.arff  
replace final-dataset.arff? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final-dataset.arff      


In [ ]:
# ! pip install liac-arff
import pandas as pd
import tensorflow as tf
# from scipy.io import arff
import arff
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# long_data = arff.loadarff('final-dataset.arff')

In [ ]:
decoder = arff.ArffDecoder()
data_file = open('final-dataset.arff', 'r')
raw_data = decoder.decode(data_file, encode_nominal=True)

In [ ]:
# separate data and labels
data = [row[0:-1] for row in raw_data['data']]
label = [row[-1] for row in raw_data['data']]

# label = [1 if row > 0 else 0 for row in label]

# 80-train, 10-validation, 10-test
train_data = np.asarray(data[0 : int(len(data) * 0.8)])
valid_data = np.asarray(data[int(len(data) * 0.8) : int(len(data) * 0.9)])
test_data = np.asarray(data[int(len(data) * 0.9) : ])

train_label = np.asarray(label[0 : int(len(label) * 0.8)])
valid_label = np.asarray(label[int(len(label) * 0.8) : int(len(label) * 0.9)])
test_label = np.asarray(label[int(len(label) * 0.9) : ])

train_label = tf.keras.utils.to_categorical(train_label, 5)
valid_label = tf.keras.utils.to_categorical(valid_label, 5)
test_label = tf.keras.utils.to_categorical(test_label, 5)

In [ ]:
input_size = len(train_data[0])
output_size = len(set(label))
hidden_layer_size = 75
ACT_FN = 'relu'

model = tf.keras.Sequential([
          tf.keras.layers.Dense(30, input_dim=input_size, kernel_initializer='uniform', activation='relu'), # input layer

          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 1st hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 2nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 3nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 4nd hidden layer
          # tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 5nd hidden layer
          # tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 6nd hidden layer
          # tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 7nd hidden layer
          # tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 8nd hidden layer   

          tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

print(model.summary())

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_124 (Dense)           (None, 30)                840       
                                                                 
 dense_125 (Dense)           (None, 75)                2325      
                                                                 
 dense_126 (Dense)           (None, 75)                5700      
                                                                 
 dense_127 (Dense)           (None, 75)                5700      
                                                                 
 dense_128 (Dense)           (None, 75)                5700      
                                                                 
 dense_129 (Dense)           (None, 5)                 380       
                                                                 
Total params: 20,645
Trainable params: 20,645
Non-tra

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# custom_opt = tf.keras.optimizers.SGD(learning_rate=0.0001) 
# model.compile(optimizer=custom_opt , loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import TensorBoard
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
max_epochs = 4
BATCH_SIZE = 2

In [ ]:
model.fit(train_data,
          train_label,
          # batch_size=BATCH_SIZE,
          epochs=max_epochs,
          # callbacks=tb,
          validation_data=(valid_data, valid_label),
          validation_steps=10,
          verbose=1
          )

Epoch 1/4
54017/54017 [==============================] - 152s 3ms/step - loss: 11.9546 - accuracy: 0.9084 - val_loss: 0.3774 - val_accuracy: 0.8966
Epoch 2/4
54017/54017 [==============================] - 151s 3ms/step - loss: 0.3863 - accuracy: 0.8960 - val_loss: 0.3776 - val_accuracy: 0.8966
Epoch 3/4
54017/54017 [==============================] - 150s 3ms/step - loss: 0.3829 - accuracy: 0.8960 - val_loss: 0.3776 - val_accuracy: 0.8966
Epoch 4/4
54017/54017 [==============================] - 151s 3ms/step - loss: 0.3797 - accuracy: 0.8960 - val_loss: 0.3775 - val_accuracy: 0.8966


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_label)

6753/6753 [==============================] - 13s 2ms/step - loss: 0.3795 - accuracy: 0.8956


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.38. Test accuracy: 89.56%
